In [40]:
import os
import pandas as pd
import numpy as np

# Set project root to ml_project directory
if os.getcwd().endswith('src'):
    os.chdir('..')
    
project_root = os.getcwd()
print(f"Working directory: {project_root}")

# Load datasets
main_df = pd.read_csv("data/learn_dataset.csv")
sport_df = pd.read_csv("data/learn_dataset_sport.csv")
job_df = pd.read_csv("data/learn_dataset_job.csv")
job_security_df = pd.read_csv("data/learn_dataset_JOB_SECURITY.csv")
retired_former_df = pd.read_csv("data/learn_dataset_retired_former.csv")
retired_jobs_df = pd.read_csv("data/learn_dataset_retired_jobs.csv")
retired_pension_df = pd.read_csv("data/learn_dataset_retired_pension.csv")
job_desc_map_df = pd.read_csv("data/code_job_desc_map.csv")
departments_df = pd.read_csv("data/departments.csv")
sports_desc_df = pd.read_csv("data/code_Sports.csv")
city_pop_df = pd.read_csv("data/city_pop.csv")
# city_revenue_df = pd.read_csv("data/city_revenue.csv", sep=';')

print(f"✓ Datasets loaded: main_df{main_df.shape}, sport_df{sport_df.shape}, job_df{job_df.shape}, job_security_df{job_security_df.shape}, retired_former_df{retired_former_df.shape}, retired_jobs_df{retired_jobs_df.shape}, retired_pension_df{retired_pension_df.shape}")

Working directory: /Users/arthur/Documents/Universite/M2-QEA/Machine Learning/final_project/ml_project
✓ Datasets loaded: main_df(50044, 10), sport_df(6460, 2), job_df(19336, 11), job_security_df(24224, 2), retired_former_df(13176, 4), retired_jobs_df(11226, 11), retired_pension_df(11226, 2)


In [41]:
job_df = job_df.merge(job_desc_map_df, left_on='job_desc', right_on='N3', how='left')
retired_jobs_df = retired_jobs_df.merge(job_desc_map_df, left_on='job_desc', right_on='N3', how='left')

# Rename columns with meaningful suffixes before merging
job_df_renamed = job_df.rename(columns={col: f"{col}_current" for col in job_df.columns if col != 'primary_key'})
retired_jobs_df_renamed = retired_jobs_df.rename(columns={col: f"{col}_retired" for col in retired_jobs_df.columns if col != 'primary_key'})

# Merge datasets
df = main_df.merge(job_df_renamed, on='primary_key', how='left')
df = df.merge(retired_jobs_df_renamed, on='primary_key', how='left')
df = df.merge(retired_pension_df, on='primary_key', how='left')
df = df.merge(sport_df, on='primary_key', how='left')
df['department'] = df['Insee_code'].str[:2]

# Save column lists after merge for use in imputers
job_cols_in_df = [col for col in df.columns if col.endswith('_current')]
retired_cols_in_df = [col for col in df.columns if col.endswith('_retired')]
pension_cols_in_df = [col for col in retired_pension_df.columns if col != 'primary_key' and col in df.columns]
sport_cols_in_df = [col for col in sport_df.columns if col != 'primary_key' and col in df.columns]

print(f"✓ Merged datasets with meaningful suffixes:")
print(f"  - Current job columns: {len(job_cols_in_df)} (suffix: _current)")
print(f"  - Retired job columns: {len(retired_cols_in_df)} (suffix: _retired)")
print(f"  - Pension columns: {len(pension_cols_in_df)}")
print(f"  - Sport columns: {len(sport_cols_in_df)}")

✓ Merged datasets with meaningful suffixes:
  - Current job columns: 13 (suffix: _current)
  - Retired job columns: 13 (suffix: _retired)
  - Pension columns: 1
  - Sport columns: 1


In [42]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.base import BaseEstimator, TransformerMixin

# Custom transformer for activity_type based imputation
class ActivityTypeImputer(BaseEstimator, TransformerMixin):
    """Impute job and retired columns with 'not_applicable' based on activity_type"""
    
    def __init__(self, retired_cols=None, job_cols=None, pension_cols=None):
        self.retired_cols = retired_cols
        self.job_cols = job_cols
        self.pension_cols = pension_cols
    
    def fit(self, X, y=None):
        # Store the retired columns if not provided
        if self.retired_cols is None:
            self.retired_cols_ = [col for col in retired_jobs_df.columns if col != 'primary_key']
        else:
            self.retired_cols_ = self.retired_cols
        
        # Store the job columns if not provided
        if self.job_cols is None:
            self.job_cols_ = [col for col in job_df.columns if col != 'primary_key']
        else:
            self.job_cols_ = self.job_cols
        
        # Store the pension columns if not provided
        if self.pension_cols is None:
            self.pension_cols_ = [col for col in retired_pension_df.columns if col != 'primary_key']
        else:
            self.pension_cols_ = self.pension_cols
        
        return self
    
    def transform(self, X):
        X_copy = X.copy()
        
        # Check if activity_type exists
        if 'activity_type' in X_copy.columns:
            # Mask for non-retired individuals (for retired job columns)
            non_retired_mask = X_copy['activity_type'] != 'type2_1'
            
            # Mask for non-employed individuals (for job columns)
            # Not employed = unemployed (type1_2) or any inactive (type2_X)
            non_employed_mask = (X_copy['activity_type'] == 'type1_2') | (X_copy['activity_type'].str.startswith('type2_'))
            
            # Fill retired job columns for non-retired people
            for col in self.retired_cols_:
                if col in X_copy.columns:
                    if X_copy[col].dtype == 'object' or X_copy[col].dtype.name == 'category':
                        # Categorical: fill with 'not_applicable'
                        X_copy.loc[non_retired_mask & X_copy[col].isna(), col] = 'not_applicable'
                    else:
                        # Numeric (hours, earnings): fill with 0
                        X_copy.loc[non_retired_mask & X_copy[col].isna(), col] = 0
            
            # Fill job columns for non-employed people (unemployed or inactive)
            for col in self.job_cols_:
                if col in X_copy.columns:
                    if X_copy[col].dtype == 'object' or X_copy[col].dtype.name == 'category':
                        # Categorical: fill with 'not_applicable'
                        X_copy.loc[non_employed_mask & X_copy[col].isna(), col] = 'not_applicable'
                    else:
                        # Numeric (hours, earnings): fill with 0
                        X_copy.loc[non_employed_mask & X_copy[col].isna(), col] = 0
            
            # Fill pension columns with 0 for non-retired people
            for col in self.pension_cols_:
                if col in X_copy.columns:
                    # Pension columns are numeric (income), fill with 0 for non-retired
                    X_copy.loc[non_retired_mask & X_copy[col].isna(), col] = 0
        
        return X_copy

# Custom transformer for sport columns imputation
class SportImputer(BaseEstimator, TransformerMixin):
    """Impute sport columns with 'not_applicable' for all missing values"""
    
    def __init__(self, sport_cols=None):
        self.sport_cols = sport_cols
    
    def fit(self, X, y=None):
        # Store the sport columns if not provided
        if self.sport_cols is None:
            self.sport_cols_ = [col for col in sport_df.columns if col != 'primary_key']
        else:
            self.sport_cols_ = self.sport_cols
        
        return self
    
    def transform(self, X):
        X_copy = X.copy()
        
        # Fill all sport columns with 'not_applicable' for missing values
        for col in self.sport_cols_:
            if col in X_copy.columns:
                # Fill missing values with 'not_applicable'
                X_copy[col].fillna('not_applicable', inplace=True)
        
        return X_copy

# Create custom transformer instances using saved column lists from merge
activity_imputer = ActivityTypeImputer(
    retired_cols=retired_cols_in_df,
    job_cols=job_cols_in_df,
    pension_cols=pension_cols_in_df
)

sport_imputer = SportImputer(
    sport_cols=sport_cols_in_df
)

# Identify column types (after activity imputation will be applied)
numeric_features = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_features = df.select_dtypes(include=['object', 'category']).columns.tolist()

# Remove target and primary_key from features
if 'target' in numeric_features:
    numeric_features.remove('target')
if 'primary_key' in numeric_features:
    numeric_features.remove('primary_key')
if 'primary_key' in categorical_features:
    categorical_features.remove('primary_key')

print(f"Initial numeric features: {len(numeric_features)}")
print(f"Initial categorical features: {len(categorical_features)}")

# Filter categorical features: keep only those with < 100 unique categories
categorical_features_filtered = []
categorical_features_dropped = []
max_categories = 100

for col in categorical_features:
    n_unique = df[col].nunique()
    if n_unique < max_categories:
        categorical_features_filtered.append(col)
    else:
        categorical_features_dropped.append((col, n_unique))

print(f"\n✓ Categorical features kept (< {max_categories} categories): {len(categorical_features_filtered)}")
print(f"✗ Categorical features dropped (≥ {max_categories} categories): {len(categorical_features_dropped)}")

if categorical_features_dropped:
    print("\nDropped features:")
    for col, n_unique in categorical_features_dropped:
        print(f"  - {col}: {n_unique} unique values")

# Update categorical features list
categorical_features = categorical_features_filtered

# Create preprocessing pipelines for numeric and categorical features
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

# Combine preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Create full pipeline with custom transformers, preprocessing, and model
full_pipeline = Pipeline(steps=[
    ('activity_imputer', activity_imputer),  # Activity-type based imputation
    ('sport_imputer', sport_imputer),         # Sport columns imputation
    ('preprocessor', preprocessor),           # Standard preprocessing (impute, scale, encode)
    ('regressor', HistGradientBoostingRegressor(random_state=42, max_iter=200))
])

print("\n✓ Pipeline created with:")
print("  1. Activity-type based imputation")
print("     - Categorical columns → 'not_applicable' for non-applicable cases")
print("     - Numeric columns (hours/earnings) → 0 for non-applicable cases")
print("     - Pension income → 0 for non-retired individuals")
print("  2. Sport columns imputation")
print("     - All sport columns → 'not_applicable' for missing values")
print("  3. Standard preprocessing (imputation, scaling, encoding)")
print(f"  4. One-hot encoding only for features with < {max_categories} categories")
print("  5. HistGradientBoostingRegressor")

# For saving preprocessed data, we need to fit_transform the preprocessing steps
X = df.drop(columns=['target', 'primary_key'], errors='ignore')
y = df['target'] if 'target' in df.columns else None

# Apply custom imputation first
X_imputed = activity_imputer.fit_transform(X)
X_imputed = sport_imputer.fit_transform(X_imputed)

# Then apply preprocessing
main_df_dummies = preprocessor.fit_transform(X_imputed)
main_df_dummies = pd.DataFrame(
    main_df_dummies,
    columns=preprocessor.get_feature_names_out()
)

# Add target back if it exists
if y is not None:
    main_df_dummies['target'] = y.values

print(f"\n✓ Preprocessed data shape: {main_df_dummies.shape}")

Initial numeric features: 5
Initial categorical features: 31

✓ Categorical features kept (< 100 categories): 26
✗ Categorical features dropped (≥ 100 categories): 5

Dropped features:
  - Insee_code: 13640 unique values
  - job_desc_current: 406 unique values
  - N3_current: 406 unique values
  - job_desc_retired: 394 unique values
  - N3_retired: 394 unique values

✓ Pipeline created with:
  1. Activity-type based imputation
     - Categorical columns → 'not_applicable' for non-applicable cases
     - Numeric columns (hours/earnings) → 0 for non-applicable cases
     - Pension income → 0 for non-retired individuals
  2. Sport columns imputation
     - All sport columns → 'not_applicable' for missing values
  3. Standard preprocessing (imputation, scaling, encoding)
  4. One-hot encoding only for features with < 100 categories
  5. HistGradientBoostingRegressor


/var/folders/ph/04p3g2s921j6d8fvnrbb7rcm0000gn/T/ipykernel_31402/2825931033.py:101: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_copy[col].fillna('not_applicable', inplace=True)



✓ Preprocessed data shape: (50044, 782)


In [48]:
# Calculate share of retired people who moved departments between job and retirement
# Previous_dep_retired: where they lived during their last job
# department: where they currently live (from Insee_code)
people_with_both = df[['department', 'Previous_dep_retired']].dropna()

if len(people_with_both) > 0:
    moved_departments = (people_with_both['department'] != people_with_both['Previous_dep_retired']).sum()
    share_moved = moved_departments / len(people_with_both) * 100
    
    print(f"Retired people with both residence info: {len(people_with_both):,}")
    print(f"Retired people who moved departments: {moved_departments:,}")
    print(f"Share who moved departments: {share_moved:.2f}%")
else:
    print("No retired people with both previous and current department information")

Retired people with both residence info: 10,894
Retired people who moved departments: 10,734
Share who moved departments: 98.53%


In [44]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# Prepare data for training
X = df.drop(columns=['target', 'primary_key'], errors='ignore')
y = df['target']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("=" * 60)
print("TRAINING HISTGRADIENTBOOSTING MODEL")
print("=" * 60)
print(f"Training set size: {len(X_train)}")
print(f"Test set size: {len(X_test)}")

# Fit the full pipeline (includes preprocessing and model)
print("\nFitting pipeline...")
full_pipeline.fit(X_train, y_train)
print("✓ Model trained successfully")

# Make predictions
y_train_pred = full_pipeline.predict(X_train)
y_test_pred = full_pipeline.predict(X_test)

# Calculate metrics
train_r2 = r2_score(y_train, y_train_pred)
train_mse = mean_squared_error(y_train, y_train_pred)
train_rmse = np.sqrt(train_mse)
train_mae = mean_absolute_error(y_train, y_train_pred)

test_r2 = r2_score(y_test, y_test_pred)
test_mse = mean_squared_error(y_test, y_test_pred)
test_rmse = np.sqrt(test_mse)
test_mae = mean_absolute_error(y_test, y_test_pred)

# Display results
print("\n" + "=" * 60)
print("MODEL PERFORMANCE")
print("=" * 60)
print(f"\n{'Metric':<30} {'Train':<15} {'Test':<15}")
print("-" * 60)
print(f"{'R² Score':<30} {train_r2:<15.4f} {test_r2:<15.4f}")
print(f"{'Mean Squared Error':<30} {train_mse:<15.4f} {test_mse:<15.4f}")
print(f"{'Root Mean Squared Error':<30} {train_rmse:<15.4f} {test_rmse:<15.4f}")
print(f"{'Mean Absolute Error':<30} {train_mae:<15.4f} {test_mae:<15.4f}")
print("=" * 60)

# Check for overfitting
if train_r2 - test_r2 > 0.1:
    print(f"\n⚠️  Warning: Possible overfitting detected (train R² - test R² = {train_r2 - test_r2:.4f})")
else:
    print(f"\n✓ Good generalization (train R² - test R² = {train_r2 - test_r2:.4f})")

TRAINING HISTGRADIENTBOOSTING MODEL
Training set size: 40035
Test set size: 10009

Fitting pipeline...


/var/folders/ph/04p3g2s921j6d8fvnrbb7rcm0000gn/T/ipykernel_31402/2825931033.py:101: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_copy[col].fillna('not_applicable', inplace=True)


✓ Model trained successfully


/var/folders/ph/04p3g2s921j6d8fvnrbb7rcm0000gn/T/ipykernel_31402/2825931033.py:101: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_copy[col].fillna('not_applicable', inplace=True)
/var/folders/ph/04p3g2s921j6d8fvnrbb7rcm0000gn/T/ipykernel_31402/2825931033.py:101: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting value


MODEL PERFORMANCE

Metric                         Train           Test           
------------------------------------------------------------
R² Score                       0.8227          0.7982         
Mean Squared Error             0.0166          0.0189         
Root Mean Squared Error        0.1290          0.1376         
Mean Absolute Error            0.0973          0.1032         

✓ Good generalization (train R² - test R² = 0.0244)
